### Overview

This notebook can be used with both a single or multi- speaker corpus and allows the interactive plotting of speaker embeddings linked to underlying audio (see instructions in the repo's speaker_embedding directory)

Depending on the directory structure used for your corpus, you may need to adjust handling of **speaker_to_utter** and **locations**.

In [1]:
import os
import glob
import random
import numpy as np
import torch
import umap

from TTS.speaker_encoder.model import SpeakerEncoder
from TTS.utils.audio import AudioProcessor
from TTS.utils.io import load_config

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, BoxZoomTool, ResetTool, OpenURL, TapTool
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Category20

For larger sets of speakers, you can use **Category20**, but you need to change it in the **pal** variable too

List of Bokeh palettes here: http://docs.bokeh.org/en/1.4.0/docs/reference/palettes.html

**NB:** if you have problems with other palettes, first see https://stackoverflow.com/questions/48333820/why-do-some-bokeh-palettes-raise-a-valueerror-when-used-in-factor-cmap

output_notebook()

You should also adjust all the path constants to point at the relevant locations for you locally

In [2]:
MODEL_RUN_PATH = "/media/alexander/LinuxFS/Documents/PycharmProjects/TTS/speaker_encoder/Trainings/"
MODEL_PATH = MODEL_RUN_PATH + "best_model.pth.tar"
CONFIG_PATH = MODEL_RUN_PATH + "config.json"

# My single speaker locations
#EMBED_PATH = "/home/neil/main/Projects/TTS3/embeddings/neil14/"
#AUDIO_PATH = "/home/neil/data/Projects/NeilTTS/neil14/wavs/"

# My multi speaker locations
EMBED_PATH = "/media/alexander/LinuxFS/Documents/PycharmProjects/TTS/speaker_encoder/Embeddings/gothic_embeddings/"
AUDIO_PATH = "/media/alexander/LinuxFS/SpeechData/GothicSpeech/NPC_Speech/"

In [3]:
!ls -1 $MODEL_RUN_PATH

best_model.pth.tar
config.json


In [4]:
CONFIG = load_config(CONFIG_PATH)
ap = AudioProcessor(**CONFIG['audio'])

> Setting up Audio Processor...
 | > sample_rate:22050
 | > num_mels:40
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > num_freq:1025
 | > power:1.5
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:40.0
 | > mel_fmax:8000.0
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:None
 | > hop_length:256
 | > win_length:1024
 | > n_fft:2048


Bring in the embeddings created by **compute_embeddings.py**

In [5]:
embed_files = glob.glob(EMBED_PATH+"/**/*.npy", recursive=True)
print(f'Embeddings found: {len(embed_files)}')

Embeddings found: 10984


Check that we did indeed find an embedding

In [6]:
embed_files[0]

'/media/alexander/LinuxFS/Documents/PycharmProjects/TTS/speaker_encoder/Embeddings/gothic_embeddings/ass_fat_hectic/wav/info_ass_fat_hectic_felipe_skalveram320060123171910_0_g3.npy'

### Process the speakers

Assumes count of **speaker_paths** corresponds to number of speakers (so a corpus in just one directory would be treated like a single speaker and the multiple directories of LibriTTS are treated as distinct speakers)

In [7]:
speaker_paths = list(set([os.path.dirname(os.path.dirname(embed_file)) for embed_file in embed_files]))
speaker_to_utter = {}
for embed_file in embed_files:
    speaker_path = os.path.dirname(os.path.dirname(embed_file))
    try:
        speaker_to_utter[speaker_path].append(embed_file)
    except:
        speaker_to_utter[speaker_path]=[embed_file]
print(f'Speaker count: {len(speaker_paths)}')

Speaker count: 18


Load embeddings with UMAP

### Set up the embeddings

Adjust the number of speakers to select and the number of utterances from each speaker and they will be randomly sampled from the corpus

In [8]:
embeds = []
labels = []
locations = []

# single speaker 
#num_speakers = 1
#num_utters = 1000

# multi speaker
num_speakers = 14
num_utters = 50


speaker_idxs = np.random.choice(range(len(speaker_paths)), num_speakers, replace=False)
print(speaker_idxs)

for speaker_num, speaker_idx in enumerate(speaker_idxs):
    speaker_path = speaker_paths[speaker_idx]
    if 'orc' in speaker_path or 'Steck' in speaker_path:
        continue
    print(speaker_path, speaker_idx)
    speakers_utter = speaker_to_utter[speaker_path]
    utter_idxs = np.random.randint(0, len(speakers_utter) , num_utters)
    for utter_idx in utter_idxs:
            embed_path = speaker_to_utter[speaker_path][utter_idx]
            embed = np.load(embed_path)
            embeds.append(embed)
            labels.append(str(speaker_num))
            locations.append(embed_path.replace(EMBED_PATH, '').replace('.npy','.wav'))
embeds = np.concatenate(embeds)

[ 8  1  6 10 17  2  5 13  4  0 16  3  9 15 14]
/media/alexander/LinuxFS/Documents/PycharmProjects/TTS/speaker_encoder/Embeddings/gothic_embeddings/Nick Benjamin 8
/media/alexander/LinuxFS/Documents/PycharmProjects/TTS/speaker_encoder/Embeddings/gothic_embeddings/Sascha Rotermund 1
/media/alexander/LinuxFS/Documents/PycharmProjects/TTS/speaker_encoder/Embeddings/gothic_embeddings/hum_fat_bearish 6
/media/alexander/LinuxFS/Documents/PycharmProjects/TTS/speaker_encoder/Embeddings/gothic_embeddings/hum_elite_hard 10
/media/alexander/LinuxFS/Documents/PycharmProjects/TTS/speaker_encoder/Embeddings/gothic_embeddings/Karl-Jürgen Sihler 17
/media/alexander/LinuxFS/Documents/PycharmProjects/TTS/speaker_encoder/Embeddings/gothic_embeddings/Rolf Birkholz 2
/media/alexander/LinuxFS/Documents/PycharmProjects/TTS/speaker_encoder/Embeddings/gothic_embeddings/ass_fat_hectic 5
/media/alexander/LinuxFS/Documents/PycharmProjects/TTS/speaker_encoder/Embeddings/gothic_embeddings/Christian Jungwirth 4
/medi

In [9]:
model = umap.UMAP()
projection = model.fit_transform(embeds)

### Interactively charting the data in Bokeh

Set up various details for Bokeh to plot the data

You can use the regular Bokeh [tools](http://docs.bokeh.org/en/1.4.0/docs/user_guide/tools.html?highlight=tools) to explore the data, with reset setting it back to normal

Once you have started the local server (see cell below) you can then click on plotted points which will open a tab to play the audio for that point, enabling easy exploration of your corpus

File location in the tooltip is given relative to **AUDIO_PATH**

In [10]:
source_wav_stems = ColumnDataSource(
        data=dict(
            x = projection.T[0].tolist(),
            y = projection.T[1].tolist(),
            desc=locations,
            label=labels
        )
    )

hover = HoverTool(
        tooltips=[
            ("file", "@desc"),
            ("speaker", "@label"),
        ]
    )

# optionally consider adding these to the tooltips if you want additional detail
# for the coordinates: ("(x,y)", "($x, $y)"),
# for the index of the embedding / wav file: ("index", "$index"),

factors = list(set(labels))
pal_size = max(len(factors), 3)
pal = Category20[pal_size]

p = figure(plot_width=1200, plot_height=800, tools=[hover,BoxZoomTool(), ResetTool(), TapTool()])


p.circle('x', 'y',  source=source_wav_stems, color=factor_cmap('label', palette=pal, factors=factors),)

url = "http://localhost:8000/@desc"
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

show(p)

### Local server to serve wav files from corpus

This is required so that when you click on a data point the hyperlink associated with it will be served the file locally.

There are other ways to serve this if you prefer and you can also run the commands manually on the command line

The server will continue to run until stopped. To stop it simply interupt the kernel (ie square button or under Kernel menu)

In [9]:
%cd $AUDIO_PATH
%pwd
!python -m http.server

/media/alexander/LinuxFS/SpeechData/GothicSpeech/NPC_Speech
Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
